In [1]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import words
from collections import Counter

In [2]:
data = pd.read_excel("Review KCU Pulau Jawa.xlsx")

In [3]:
data = pd.read_excel("Review KCU Pulau Jawa.xlsx")
data = data.dropna(subset=['review_text'])

data['review_text_cleaned'] = data['review_text'].astype(str).str.lower()
data['review_text_cleaned'] = [re.sub(r'[^\x00-\x7f]',r'', i) for i in data['review_text_cleaned']]
data['review_text_cleaned'] = [re.sub(r'\n', r' ', i) for i in data['review_text_cleaned']]
data['review_text_cleaned'] = data['review_text_cleaned'].apply(
    lambda x: re.sub(f"[{re.escape(string.punctuation)}]", " ", x)
)
# Remove numbers
data['review_text_cleaned'] = data['review_text_cleaned'].apply(lambda x: re.sub(r'\d+', '', x))

# Remove English words (basic filtering using a set of common English words)
nltk.download('words')
english_words = set(words.words())

def remove_english_words(text):
    return ' '.join([word for word in text.split() if word not in english_words])

data['review_text_cleaned'] = data['review_text_cleaned'].apply(remove_english_words)

# Normalize whitespace again after filtering
data['review_text_cleaned'] = data['review_text_cleaned'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# Drop empty rows again
data = data[data['review_text_cleaned'] != ""]

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\POS\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [4]:
all_words = ' '.join(data['review_text_cleaned']).split()
word_freq = Counter(all_words)

In [5]:
def has_repeated_letters(word):
    return re.search(r'(.)\1{2,}', word) is not None

suspect_words = [
    w for w in word_freq
    if word_freq[w] < 5 or has_repeated_letters(w)
]
suspect_words

['cwus',
 'displaying',
 'parcels',
 'features',
 'december',
 'attempted',
 'memangnya',
 'sebatas',
 'tuliskan',
 'ditempelkan',
 'tolonglah',
 'cb',
 'jpg',
 'jakpus',
 'sminggu',
 'tlng',
 'darri',
 'sekaliini',
 'borok',
 'hoak',
 'sekalinya',
 'dijawb',
 'catatan',
 'terterah',
 'bunyi',
 'gedoran',
 'pendapat',
 'menjd',
 'setannya',
 'ketularan',
 'sangaaattt',
 'sangaat',
 'kecewaaa',
 'ngmong',
 'sebutkan',
 'knpa',
 'skarang',
 'brang',
 'kmna',
 'brsangkutan',
 'brsikap',
 'bodo',
 'cn',
 'ktsh',
 'idjkta',
 'jakartasoekarnohatta',
 'reaksinya',
 'tindaklanjutin',
 'pemeriksaan',
 'blg',
 'tersangkut',
 'dibayarkan',
 'bensin',
 'tugasnya',
 'ind',
 'customernya',
 'menanggapinya',
 'sebab',
 'kepentingan',
 'meyakinkan',
 'terusan',
 'nlpn',
 'kejauhan',
 'musim',
 'ujan',
 'yos',
 'sudarso',
 'aturannya',
 'yng',
 'lntai',
 'kapanpun',
 'pemerintag',
 'sebagus',
 'reformasi',
 'membenci',
 'mamah',
 'pki',
 'thailand',
 'lmbat',
 'dijakarta',
 'managemen',
 'fakta',
 'diu

In [6]:
df_suspect = pd.DataFrame(suspect_words, columns=['suspect_word'])
df_suspect.to_excel('suspect_words.xlsx', index=False)